# <center>Wrangle Report</center>

This document outlines the gather, assessment and wrangle stages of tweet data for `WeRateDog` twitter handle. 

## <center>Gather data sources</center>

The project deals there data sources. These are: 

<ol>
    <li>Twitter archive of tweets from WeRateDogs Twitter handle. Tweets have been analyzed and enhanced. The enhanced archive is available <a href="https://d17h27t6h515a5.cloudfront.net/topher/2017/August/59a4e958_twitter-archive-enhanced/twitter-archive-enhanced.csv">here</a></li>
    <li>Every dog image of WeRateDogs Twitter archive has been analyzed through a neural network that can classify breeds of dogs. That image prediction file is available <a href="https://s3.amazonaws.com/video.udacity-data.com/topher/2018/November/5bf60c69_image-predictions-3/image-predictions-3.tsv">here</a></li>
    <li>Finally each tweet's retweet count and link count are metrics of interest. That data is retrieved using Twitter's API
</ol>

Data downloaded from these sources will be stored in `data` directory. The first step is to create the directory if one such doesn't exist.<br>

Twitter archive tweets are downloaded using Python's `request` object. The same method is used for downloading dog breed prediction file.<br>

To increase code reusability I factored common functionalities in Python functions like `dowload_file`
Downloading data from Twitter requires using Twitter provided API. The prerquisite for the API call is to set Twitter developer account and making API call after acquiring access token. Downloading data using Twitter's API call requires tweet IDs. Individual tweet IDs are obtained from Twitter archive dump.<br>

To improve the performance to data download from Twitter, I use Twitter's batch API. This batch API requires a batch of 100 tweet IDs per call. In less than 1 minute, 2500+ tweets get downloaded.<br>

While downlading retweet & like counts per tweet, I also cleanse and extract `source` field.<br>

<b>16</b> tweets couldn't be downloaded even after making a separate attempt.


## <center>Assessment</center>

My assessment consists of two parts. The first part is visual assessment which involved opening up `csv` file in Google sheet as well as using some Pandas functions like `info()` to see schema, data type or `value_counts()` to observe distinct values of a specific columns.<br>

The subsequent assessment step involves programatic assessment of data.

Three data sources described above are loaded into three dataframes

<ol>
    <li>twitter_enhanced_df: Enhanced twitter archive</li>
    <li>tweet_data_df: Some metrics like retweet count , favorite count about every tweet</li>
    <li>prediction_df: classification of dogs by tweet ID using neural network</li>
</ol>

### <center>Cleanliness Issues</center>

<ol>
    <li>`timestamp` column values should not have trailing +00000</li>
    <li>`rating_denominator` is zero for tweet ID = 835246439529840640. That needs to be removed</li> 
    <li>For few records rating_numerator & ratings denominator contain incorrect rating values. These happened for where the ratings numerator or denominator in original text had floating point numbers</li>
    <li>`source` can't used for data analysis purpose as it contains html text with "a href" string</li>
    <li>Some records correspond to retweet. They need to be removed</li>
    <li>Some records correspond to replies. They need to be removed</li>
    <li>Some tweets have objects other than dog</li>
    <li>Not all tweets have related dog picture</li>
    <li>`ratings_numerator` contains incorrect value specially when the text containing the rating uses floating point number for ratings</li>
    <li>There are tweets with multiple dog stages for a single dog e.g. doggo & pupper</li>
</ol>


### <center>Structural Issues</center>

<ol>
    <li>Instead of 4 `dog_stage` fields in `twitter_enhanced_df`, use only one column for categorical values</li>
    <li>All three dataframes need to be combined together for ease of analysis</li>
    <li>Use only one dog classification form `prediction_df` with highest confidence value</li>
    <li>Change `timestamp` column in `twitter_enhanced_df` to datetime data type</li>
    <li>Change `rating` fields in `twitter_enhanced_df` to float data type</li>
</ol>


## <center>Data Wrangling</center>

`twitter_enhanced_df` is copied to `cleaned_twitter_df` before data wrangling.

<b> Issue 1: timestamp column values should not have trailing +00000</b>

Trim the trailing zero of the timestamp column in `twitter_enhanced_df`

<b> Issue 2: One tweet has `rating_denominator=0` in `cleaned_twitter_df`</b>

Delete the record with `rating_denominator=0` from `cleaned_twitter_df`

<b> Issue 3: rating_numerator values are incorrect when tweet text uses floating point numbers</b>

`rating_num` and `rating_den` are added to the dataframe during assessment stage by parsing the tweet text. These fields are compared to `rating_numerator` & `rating_denominator` respectively to determined that derived fields are more accurate specially when text uses floating point numbers as rating. 

New field `rating` is computed using `rating_num` / `rating_den`

`rating_numerator` & `rating_denominator` fields are dropped.

<b> Issue 4: `source` field in `cleaned_twitter_df` can't be used for data analysis purpose as it contains html string</b>

Drop `source` field from `twitter_enhanced_df`. Cleaned `source` column is included in `tweet_data_df`

<b> Issue 5: `timestamp` field has string as its data type</b>

Changed data type to `datetime` using `pd.to_datetime` field

<b> Issue 6: There are 4 `dog_stage` fields in `cleaned_enhanced_df`</b>

`dog_stage` can be considered as categorical variable. Combine 4 fields in one field `dog_stage` and 4 fields.

<b> Issue 7: Some records correspond to retweet. They need to be removed</b>

Retweet in `cleaned_twitter_df` is determined by `retweeted_status_id` > 0. These records are deleted.

<b> Issue 8: Some records correspond to replies. They need to be removed</b>

Replies in `cleaned_twitter_df` is determined by `in_reply_to_status_id` > 0. These records are deleted.

<b> Issue 9: `prediction_df` contains images other than dog</b>

These records in `prediction_df` are not useful. They are identified where `p1_dog == False and p2_dog == False and p3__dog == False`

<b> Issue 10: Not all tweets have related dog picture</b>

Join `cleaned_twitter_df` with `cleaned_prediction_df`. Only matching records relate to tweets having dog images. Removed remaining records.

<b> Issue 11: All three dataframes need to be combined together for ease of analysis</b>

Join three dataframes to produce a single dataframe. This dataframe will be used for subsequent analysis



## <center>Additional Data Wrangling</center>

While doing visual assessment of tweet text, I noticed that dog's geneder can be derived from related pronouns. Pronouns like `he/his` can be used to assihgn `male` while `she/her` for female gender.

`gender` field is also added in the final dataframe